## Arranging the data - Create an utility matrix

In [6]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sortedcontainers
from collections import Counter



### Creating a DataFrame with cities that belonging to all methods

In [ ]:
import itertools
def create_DataFrame_with_filt_cities(data,all_similarities_cities_lst_str):

    def from_iterable_unique(iterables): # I edited the function chain.from_iterable
        for it in iterables.str.split(', '):
            c1=Counter(it)
            set_it= set(it)
            for _ in set_it:
                yield c1.popitem()

    def unique_searches_counter(iterables): # I edited the function chain.from_iterable
        for it in iterables.str.split(', '):
            c1=Counter(it)         
            set_it= set(it)
            cnt=0 #count the searces in each session (last->first)
            for _ in set_it:
                c1.popitem()
                cnt+=1
                yield cnt 

    # calculate the amount of different cities in each session
    # data2=data.copy()
    # data2['cityCodes'].str.split(', ').apply(sortedcontainers.SortedSet).apply(lambda x: rem_fw(x, all_similarities_cities))
    data['cityCodes_filt']=data.apply(lambda x: [x for x in list(x['cityCodes'].split(', ')) if x in all_similarities_cities_lst_str], axis=1)
    data['cityCodes_filt2']=data.apply(lambda x: list(dict.fromkeys([x for x in list(x['cityCodes'].split(', ')) if x in all_similarities_cities_lst_str])), axis=1)
    data['n_cityCodes_filt2']=data.apply(lambda x: len(x['cityCodes_filt2']), axis=1)

    lens = data['cityCodes'].str.split(', ').apply(sortedcontainers.SortedSet).map(len)

    # create new dataframe, repeating or chaining as appropriate
    res = pd.DataFrame({'userId': np.repeat(data['userId'], lens),
                        'sessionId': np.repeat(data['sessionId'], lens),
                        'unique_cityCodes': np.repeat(data['cityCodes'], lens),
                        'unique_cityCodes_filt': np.repeat(data['cityCodes_filt'], lens),
                        'unique_cityCodes_filt2': np.repeat(data['cityCodes_filt2'], lens),
                        'nunique_cityCodes_filt2': np.repeat(data['n_cityCodes_filt2'], lens),
                        'city_searches': from_iterable_unique(data['cityCodes']),
                        'search_cnt': unique_searches_counter(data['cityCodes']), #From the end to the beginning of the session
                        'n_unique_cities': lens}) # Num of different cities in  each session

    res['city'], res['searches'] = zip(*res.city_searches)
    #res[res['unique_cityCodes_filt'].str.split(', ').apply(sortedcontainers.SortedSet).apply(lambda x: rem_fw(x, all_similarities_cities))]
    return res




### Creating a DataFrame

In [7]:
import itertools
def create_DataFrame(data):

    def from_iterable_unique(iterables): # I edited the function chain.from_iterable
        for it in iterables.str.split(', '):
            c1=Counter(it)
            set_it= set(it)
            for _ in set_it:
                yield c1.popitem()

    def unique_searches_counter(iterables): # I edited the function chain.from_iterable
        for it in iterables.str.split(', '):
            c1=Counter(it)         
            set_it= set(it)
            cnt=0 #count the searces in each session (last->first)
            for _ in set_it:
                c1.popitem()
                cnt+=1
                yield cnt 

    # calculate the amount of different cities in each session
    lens = data['cityCodes'].str.split(', ').apply(set).map(len)
    # create new dataframe, repeating or chaining as appropriate
    res = pd.DataFrame({'userId': np.repeat(data['userId'], lens),
                        'sessionId': np.repeat(data['sessionId'], lens),
                        'unique_cityCodes': np.repeat(data['cityCodes'], lens),
                        'city_searches': from_iterable_unique(data['cityCodes']),
                        'search_cnt': unique_searches_counter(data['cityCodes']), #From the end to the beginning of the session
                        'n_unique_cities': lens}) # Num of different cities in  each session

    res['city'], res['searches'] = zip(*res.city_searches)
    return res


### Filter sessions that have at least N searches of different cities

In [8]:

def filter_sessions_N_dif_searches(data, required_searches):
    return data.loc[(data['nunique_cityCodes_filt2'] >= required_searches)]

def sessions_without_last(data):
    return data.loc[(data['search_cnt'] > 1)]
    
def sessions_with_only_last(data):
    return data.loc[(data['search_cnt'] == 1)]


### Transforming the data



In [9]:
def transforming_data (res):
    utility_matrix= res.pivot_table(index='sessionId',columns = 'city', values='searches', aggfunc=np.sum, fill_value=0) #it also merges the sessionId
    return utility_matrix


### remove unknown sessions

In [10]:
def remove_unknown_sessions (utility_matrix , unknown_sessions_list=[] ):
    utility_matrix=utility_matrix.drop(unknown_sessions_list, axis=0, errors='ignore') # drop the sessions in the list if they exist
    return utility_matrix
